In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy import io
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Create an empty 2D NumPy array with a shape of (999, 3)
rows = 10344
cols = 3
col =1    
preproccesed_header_data = np.zeros((rows, cols))        #preprccessed_header_data = [Age,Male,Female]

lable_data = np.empty(rows, dtype='<U1000') 
age_data = np.empty(rows)
sex_data = np.empty(rows)

 

def parse_header(file_path):
    header_info = {'Age': None, 'Sex': None, 'Dx': None, 'Rx': None, 'Hx': None}
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            f_name = line
            if line.startswith('# Age:'):
                age_value = line.split(': ')[-1]
                if age_value == 'NaN' or int(age_value) <= 0 or int(age_value) > 100 :
                    #header_info['Age'] = None
                    preproccesed_header_data[i][0] = mean_age/100
                    age_data[i] = mean_age/100              ########
                else :
                    preproccesed_header_data[i][0] = (int(age_value))/100
                    age_data[i]=(int(age_value)/100)
            elif line.startswith('# Sex:'):
                sex_type = line.split(': ')[-1]
                if sex_type == 'Male' or sex_type == 'M':
                    preproccesed_header_data[i][1]=1
                    sex_data[i] = 1

                elif sex_type == 'Female' or sex_type == 'F':
                    preproccesed_header_data[i][2]=1
                    sex_data[i] = 0
                else:    
                    preproccesed_header_data[i][1]=1
                    sex_data[i] = 1

            elif line.startswith('# Dx:'):
                dx_value = line.split(': ')[-1]
                header_info['Dx'] = dx_value
                lable_data[i] = dx_value
              
    return header_info


#root_folder = 'C:/Users/rwkos/Desktop/DSE PROJECT/classification-of-12-lead-ecgs-the-physionetcomputing-in-cardiology-challenge-2020-1.0.2/training/georgia'
root_folder = 'C:/Users/rwkos/Desktop/MVP Code/Final data Georgia'

# Iterate through .hea files and parse header information
data = []
"""for file_name in os.listdir(folder_path):
    if file_name.endswith('.hea'):
        file_path = os.path.join(folder_path, file_name)
        header_info = parse_header(file_path)
        data.append(header_info)"""
data_paths = []
ecg_array = []        
i=0
for root, _, files in os.walk(root_folder):
    #print(type(root))
    parts = root.split("/")
    # Get the last element from the list of parts
    last_part = parts[-1]
    output_str = last_part.split('\\')[-1]
    print(last_part)
    
    if last_part == "CNN\cpsc_2018":
        mean_age = 64
    elif last_part == "training\cpsc_2018_extra":
        mean_age = 65
    elif last_part == "Final data Georgia":
        mean_age = 62
    elif last_part == "training\ptb":
        mean_age = 56.3
    elif last_part == "training\ptb-xl":
        mean_age = 61

    for file_name in files:
        if file_name.endswith('.hea'):
            file_path = os.path.join(root, file_name)
            header_info = parse_header(file_path)
            i=i+1
            #data.append(header_info)
        elif file_name.endswith('.mat'):
            #print(str(os.getcwd()))
            #data_paths.append(os.path.join(folder_path, file_name))
            #data_paths.append('C:/Users/rwkos/Desktop/CNN/cpsc_2018/g1/'+file_name)
            data = io.loadmat(root_folder +'/'+ output_str +'/'+ file_name)  # Load .mat file
            array = data['resampled_data']         # Replace 'array' with the actual variable name in your .mat file
            ecg_array.append(array)
            #print(file_name)
            data_paths.append(root_folder+file_name)
X_array = np.stack(ecg_array, axis=0)        
Y_array = np.array([item.split(',')[0] for item in lable_data]) 


Final data Georgia
Final data Georgia\g10_final
Final data Georgia\g11_final
Final data Georgia\g1_final
Final data Georgia\g2_final
Final data Georgia\g3_final
Final data Georgia\g4_final
Final data Georgia\g5_final
Final data Georgia\g6_final
Final data Georgia\g7_final
Final data Georgia\g8_final
Final data Georgia\g9_final


In [7]:
#label encoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the label encoder on the string array
Y_array = label_encoder.fit_transform(Y_array)

#Train test split
X_train,X_val,Y_train, Y_val,age_train,age_val,sex_train,sex_val = train_test_split(
    X_array,Y_array,age_data,sex_data, test_size=0.2, random_state=42
)


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Assuming you have already loaded your data and labels into numpy arrays
# X_train: 2D numpy array of shape (num_samples, num_leads, num_data_points)
# y_train: 1D numpy array of labels

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(Y_train, dtype=torch.long)

##   Age and Sex
age_train_tensor = torch.tensor(age_train, dtype=torch.float32)
sex_train_tensor = torch.tensor(sex_train, dtype=torch.float32)

# Define a multi-input CNN model
class MultiInputECGClassifier(nn.Module):
    def __init__(self, num_classes):
        super(MultiInputECGClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_leads, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=4)
        self.fc1 = nn.Linear(16 * ((num_data_points - 4) // 4) + 2, 128)  # Adding 2 for age and sex
        self.dropout1 = nn.Dropout(p=0.6)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, ecg, age, sex):
        ecg_features = self.pool(torch.relu(self.conv1(ecg)))
        ecg_features = ecg_features.view(ecg_features.size(0), -1)  # Flatten the tensor
        age = age.unsqueeze(1)  # Expand dimensions to match ecg_features
        sex = sex.unsqueeze(1)  # Expand dimensions to match ecg_features
        combined_features = torch.cat((ecg_features, age, sex), dim=1)
        combined_features = torch.relu(self.fc1(combined_features))
        combined_features = self.dropout1(combined_features)
        output = self.fc2(combined_features)
        return output

# Initialize multi-input model
num_classes = len(set(Y_array))
num_leads = X_train.shape[1]
num_data_points = X_train.shape[2]
model = MultiInputECGClassifier(num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Create a DataLoader for batch training
batch_size = 16
train_dataset = TensorDataset(X_train_tensor, age_train_tensor, sex_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

losses = []
validation_losses = []

# Convert validation numpy arrays to PyTorch tensors
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
age_val_tensor = torch.tensor(age_val, dtype=torch.float32)
sex_val_tensor = torch.tensor(sex_val, dtype=torch.float32)
y_val_tensor = torch.tensor(Y_val, dtype=torch.long)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for inputs_ecg, inputs_age, inputs_sex, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs_ecg, inputs_age, inputs_sex)
        loss = criterion(outputs, labels)
        loss.backward()
        
        running_loss += loss.item()
        optimizer.step()

    # print(running_loss, X_train.shape[0])
    losses.append(running_loss / X_train.shape[0])

    with torch.no_grad():
        val_outputs = model(X_val_tensor, age_val_tensor, sex_val_tensor)
        loss = criterion(val_outputs, y_val_tensor)
        # print(loss.item(), X_val.shape[0])
        validation_losses.append(loss.item() / X_val.shape[0])

    print(".", end="")

    if epoch % 10 == 9:
        model.eval()
        with torch.no_grad():
            print("\nEpoch {}".format(epoch+1))
            train_outputs = model(X_train_tensor, age_train_tensor, sex_train_tensor)
            predicted_train_labels = train_outputs.argmax(dim=1).numpy()
            train_acc = accuracy_score(y_train_tensor, predicted_train_labels)
            print(f"Train Accuracy: {train_acc:.8f}")

            val_outputs = model(X_val_tensor, age_val_tensor, sex_val_tensor)
            predicted_labels = val_outputs.argmax(dim=1).numpy()
            accuracy = accuracy_score(Y_val, predicted_labels)
            print(f"Validation Accuracy: {accuracy:.8f}")

    # print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
# Assuming you have loaded your validation data, labels, age, and sex inputs
# X_val: 2D numpy array of ECG data for validation
# age_val: 1D numpy array of ages for validation
# sex_val: 1D numpy array of sexes (0 or 1) for validation
# y_val: 1D numpy array of labels for validation

# Set the model to evaluation mode
model.eval()

# Disable gradient calculations since you're not training
with torch.no_grad():
    # Forward pass on validation data
    val_outputs = model(X_val_tensor, age_val_tensor, sex_val_tensor)
    # Get predicted class labels
    predicted_labels = val_outputs.argmax(dim=1)

# Convert predicted labels to numpy array
predicted_labels_np = predicted_labels.numpy()

# Calculate accuracy
accuracy = accuracy_score(Y_val, predicted_labels_np)
print(f"Validation Accuracy: {accuracy:.4f}")

plt.plot(losses, label="train loss")
plt.legend()
plt.show()

plt.plot(validation_losses, label="validation_losses")
plt.legend()
plt.show()


..........
Epoch 10
Train Accuracy: 0.82525680
Validation Accuracy: 0.24504592
.

KeyboardInterrupt: 